# 6. 나는 SOLO -- 5의 다른구현 (torch)

## A. 임베딩레이어 이용 

In [172]:
#df_view = pd.read_csv('https://raw.githubusercontent.com/guebin/DL2024/main/posts/solo.csv',index_col=0)
df = df_view.stack().reset_index().set_axis(['W','M','y'],axis=1)
w = {'옥순':0, '영자':1, '정숙':2, '영숙':3, '순자':4, '현숙':5}
m = {'영식':0, '영철':1, '영호':2, '광수':3, '상철':4, '영수':5}
X1 = torch.tensor(df['W'].map(w)) # length-n vector 
X2 = torch.tensor(df['M'].map(m)) # length-n vector 
# E1 = torch.nn.functional.one_hot(X1).float()
# E2 = torch.nn.functional.one_hot(X2).float()
y = torch.tensor(df['y']).float().reshape(-1,1)

In [178]:
#--#
torch.manual_seed(43052)
e1 = torch.nn.Embedding(num_embeddings=6,embedding_dim=2)
b1 = torch.nn.Embedding(num_embeddings=6,embedding_dim=1)
e2 = torch.nn.Embedding(num_embeddings=6,embedding_dim=2)
b2 = torch.nn.Embedding(num_embeddings=6,embedding_dim=1)
sig = torch.nn.Sigmoid()
loss_fn = torch.nn.MSELoss()
optimizr = torch.optim.Adam(
    list(e1.parameters())+list(e2.parameters())+list(b1.parameters())+list(b2.parameters()),
    lr = 0.01
)
#--#
for epoc in range(300):
    ## step1 
    W_features = e1(X1) 
    W_bias = b1(X1)
    M_features = e2(X2) 
    M_bias = b2(X2)
    dot = (W_features * M_features + W_bias + M_bias).sum(axis=1)
    yhat = (sig(dot)*5).reshape(-1,1)
    ## step2 
    loss = loss_fn(yhat,y)
    ## step3 
    loss.backward()
    ## step4 
    optimizr.step()
    optimizr.zero_grad()

In [179]:
df_ebdspace = pd.DataFrame(torch.concat([W_features.data,W_bias.data, M_features.data, M_bias.data],axis=1))
df_ebdspace.columns = ['여성잠재특징1','여성잠재특징2','여성바이어스','남성잠재특징1','남성잠재특징2','남성바이어스']
df_yhat = pd.DataFrame(yhat.data,columns=['yhat'])
pd.concat([df,df_yhat,df_ebdspace],axis=1)

,W,M,y,yhat,여성잠재특징1,여성잠재특징2,여성바이어스,남성잠재특징1,남성잠재특징2,남성바이어스
0,옥순,영식,0.79,0.780502,-1.222563,-0.272170,-1.045559,1.373276,-1.022922,0.902046
1,옥순,영철,0.81,0.047810,-1.222563,-0.272170,-1.045559,1.030008,0.553320,-0.569689
2,옥순,광수,0.20,0.276751,-1.222563,-0.272170,-1.045559,0.918893,0.133602,0.206875
3,옥순,상철,0.21,0.351969,-1.222563,-0.272170,-1.045559,-0.054492,0.705167,-0.182116
4,영자,영식,0.81,0.629324,-1.804423,0.426392,-0.413990,1.373276,-1.022922,0.902046
5,영자,영호,0.83,0.786616,-1.804423,0.426392,-0.413990,0.372022,-0.319785,-0.021331
6,영자,광수,0.21,0.588011,-1.804423,0.426392,-0.413990,0.918893,0.133602,0.206875
7,영자,영수,0.19,0.160942,-1.804423,0.426392,-0.413990,0.826127,1.568511,-0.876783
8,정숙,영철,0.95,0.954222,1.572679,0.054415,-0.977568,1.030008,0.553320,-0.569689
9,정숙,영호,0.98,0.965395,1.572679,0.054415,-0.977568,0.372022,-0.319785,-0.021331


## B. net이용 

# 7. 나는 SOLO -- 5의 다른구현 (fastai)

## A. 코딩.. 

In [800]:
dls = fastai.collab.CollabDataLoaders.from_df(df,bs=2,valid_pct = 2/24)
lrnr = fastai.collab.collab_learner(dls, n_factors=2, y_range=(0,5))
lrnr.fit(30,lr=0.1)

epoch,train_loss,valid_loss,time
0,4.506458,2.765290,00:00
1,3.898577,3.248128,00:00
2,3.067885,1.962692,00:00
3,2.236785,0.008190,00:00
4,1.619861,0.099657,00:00
5,1.221960,0.093989,00:00
6,0.933670,0.096321,00:00
7,0.721858,0.027466,00:00
8,0.564464,0.054740,00:00
9,0.440905,0.032084,00:00


`-` 예측 

In [806]:
df_test = pd.DataFrame({'W':['정숙']*6, 'M':['영식','영철','영호','광수','상철','영수']})
df_test

,W,M
0,정숙,영식
1,정숙,영철
2,정숙,영호
3,정숙,광수
4,정숙,상철
5,정숙,영수


In [819]:
yhat,_ = lrnr.get_preds(dl = dls.test_dl(df_test))
df_test.assign(yhat = yhat)

,W,M,yhat
0,정숙,영식,4.755944
1,정숙,영철,4.883018
2,정숙,영호,4.799464
3,정숙,광수,1.065324
4,정숙,상철,0.853418
5,정숙,영수,1.410224


- 적절해보임 

## B. `fastai` 의 net, loss 등을 구경 

In [823]:
lrnr.model

EmbeddingDotBias(
  (u_weight): Embedding(7, 2)
  (i_weight): Embedding(7, 2)
  (u_bias): Embedding(7, 1)
  (i_bias): Embedding(7, 1)
)

In [824]:
lrnr.forward?? 

Signature: lrnr.forward(x)
Docstring:
Define the computation performed at every call.

Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.
Source:   
    def forward(self, x):
        users,items = x[:,0],x[:,1]
        dot = self.u_weight(users)* self.i_weight(items)
        res = dot.sum(1) + self.u_bias(users).squeeze() + self.i_bias(items).squeeze()
        if self.y_range is None: return res
        return torch.sigmoid(res) * (self.y_range[1]-self.y_range[0]) + self.y_range[0]
File:      ~/anaconda3/envs/dl2024/lib/python3.11/site-packages/fastai/collab.py
Type:      method

- 우리가 만든 모델이랑 같음!

In [826]:
lrnr.loss_func 

FlattenedLoss of MSELoss()